In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pysnobal.pysnobal import PySnobal

In [ ]:
# run PySnobal
status = PySnobal('../pysnobal/tests/pysnobal_config.ini').run()

In [ ]:
# load in the outputs
file_name = '../pysnobal/tests/test_data_point/gold_ipw/gold.snobal.out'
# file_name = '../pysnobal/tests/test_data_point/gold_ipw/gold.snobal.out.all'

output_label = np.array(['time_s','R_n','H','L_v_E','G','M','delta_Q','G_0','delta_Q_0',
            'cc_s_0','cc_s_l','cc_s','E_s','melt','ro_predict','z_s_0','z_s_l',
            'z_s','rho','m_s_0','m_s_l','m_s','h2o','T_s_0','T_s_l','T_s'])
gold = pd.read_csv(
    file_name,
    delimiter=" ", names=output_label)
start_date = pd.to_datetime('10-01-1983 00:00')
tdelta = pd.to_timedelta(gold.time_s, unit='hour')
gold['date_time'] = start_date + tdelta
gold['date_time'] = gold['date_time'].dt.round('min') # Snobal uses decimals for minutes but pandas will convert with microseconds
gold.set_index('date_time', inplace=True)
gold.index = gold.index.tz_localize('MST')
gold.drop('time_s', axis=1, inplace=True)


new = pd.read_csv(
    '../pysnobal/tests/output/pysnobal_output.csv',
    index_col='date_time', parse_dates=True)

gold = gold[new.index[0]:new.index[-1]]

# TODO fix this, it's when in the small time step
new = new.loc[~new.index.duplicated()]

print(gold.shape)
print(new.shape)

In [ ]:

# Compare the snowpack state variables
state_vars = ['cc_s_0','cc_s_l','cc_s', 'z_s_0','z_s_l',
            'z_s','rho','m_s_0','m_s_l','m_s','h2o','T_s_0','T_s_l','T_s']
# state_vars = ['cc_s_0','cc_s_l']
ncols = len(state_vars)

fig, ax = plt.subplots(ncols, 1, figsize=(15, 5*ncols))

for idx, column in enumerate(state_vars):

    par1 = ax[idx].twinx()
    d = gold[column] - new[column]
    par1.plot(new.index, np.zeros_like(new.index), 'k--')
    par1.plot(d.index, d, 'k')

    ax[idx].plot(new.index, new[column], '-')
    ax[idx].plot(gold.index, gold[column], '-')
    ax[idx].set_title(column)
    ax[idx].legend(['pysnobal', 'snobal'])


In [ ]:

# Compare the emergy balance
eb_vars = ['R_n','H','L_v_E','G','M','delta_Q','G_0','delta_Q_0',
            'E_s','melt','ro_predict']
ncols = len(eb_vars)

fig, ax = plt.subplots(ncols, 1, figsize=(15, 5*ncols))

for idx, column in enumerate(eb_vars):

    par1 = ax[idx].twinx()
    d = gold[column] - new[column]
    par1.plot(new.index, np.zeros_like(new.index), 'k--')
    par1.plot(d.index, d, 'k')

    ax[idx].plot(gold.index, gold[column])
    ax[idx].plot(new.index, new[column])
    ax[idx].set_title(column)
    ax[idx].legend(['snobal', 'pysnobal'])

    


In [ ]:
# look at the raw data differences
df = gold - new
df = df.loc[(df!=0).any(axis=1)]
df.index = df.index.tz_convert('MST')
df




In [ ]:
# print(gold.shape)
# print(new.shape)
# gold
date_val = '1983-11-12 23:59'

df = pd.DataFrame()
df['gold'] = gold.loc[date_val, :]
df['new'] = new.loc[date_val, :]
df

new


In [ ]:
# date_time differences
print(gold.index.difference(new.index))
print(new.index.difference(gold.index))
